# Introduction to Python for astronomy

Gilles Landais : gilles.landais@unistra.fr

##  lesson 4:  
- Numpy library (part2)
- Astropy library
-----------------------------------------------------------------

### Dedicated libraries for astronomy
- astropy 
- PyFITS
- astroLib
- astroquery
- ...

See also *Python for Astronomers* : http://www.astropython.org/

AstroPython is a *community for performing astronomy research with Python*. It includes tutorial, list of resources (packages), blogs.. 




----------------------------------------- 
## Numpy library (part 2)
![Numpy logo](http://www.numpy.org/_static/numpy_logo.png)

### Select a subpart of a nd-array (table)
Numpy provides a **powerful** function to extract a subpart of a table - *numpy.where* – according to constraints.

In [1]:
import numpy as np

In [2]:
# initialize a Numpy table (nd-array)
mytable = np.array([(1.,2.,3.),(0.,1.1,2.2),(-1.,-2.,-3.),(0.1,1.2,2.5)], 
                 dtype=[('col1','f8'),('col2','f8'),('col3','f8')])

# get the lines number which matches a constraint
nindice = np.where(mytable['col1']>0)

# Create a new table including lines that matches the constraint
newtable = mytable[nindice]
print (newtable)

[(1. , 2. , 3. ) (0.1, 1.2, 2.5)]


**Note:** *numpy.where* selection is a *slicing* approach. 

Tables generated by *numpy.where* are built with a copy of the values.

In [3]:
mytable[0][0] = 999
print (mytable)
print (newtable) # newtable is not changed


[( 9.99e+02,  2. ,  3. ) ( 0.00e+00,  1.1,  2.2) (-1.00e+00, -2. , -3. )
 ( 1.00e-01,  1.2,  2.5)]
[(1. , 2. , 3. ) (0.1, 1.2, 2.5)]


### Numpy serialisation with pickle

In [4]:
import numpy as np
# write mytable into a file
np.savetxt('data.txt', mytable)

In [5]:
# get table from serialisation into a Numpy structure
b = np.loadtxt('data.txt')
print (type(b))

<class 'numpy.ndarray'>


In [6]:
# write mytable into a binay file
np.save('data.npy', mytable)

# get table from binary serialisation into a Numpy structure
b = np.load('data.npy')

**Note:** the *.npy* extension is required for binary serialisation!

**Note:** the file *data.npy* contains data  **and** the numpy structure. This is a serialisation and **not** the table output.

*see: cat data.npy*

### Open a resource via URL

The following example queries a table coming from the VizieR service (see catalogues details http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=2220/stars) 

In [7]:
url = 'http://vizier.u-strasbg.fr/viz-bin/asu-tsv?-source=2220/stars&-oc.form=dec&-out=Vmag,HR,HD&-out.add=_RA,_DE'

# get table using URL 
# parameters explanations:
#   skip_heeader : skip the first lines which are comments
#   delimiter: the table returned by the URL is a TSV files (with '\t' as column delimiter)
data = np.genfromtxt(url, skip_header=34, delimiter='\t')

# print the 2 first lines
print (data[:2]) 

[[2.5914167e+01 5.0688889e+01 4.0700000e+00 4.9600000e+02 1.0516000e+04]
 [6.2164583e+01 4.7713056e+01 4.0400000e+00 1.2730000e+03 2.5940000e+04]]


The same function with column name and type specified (dtype structure - see Numpy part1)

In [8]:
# get table ans specify the name+type for the 4 columns returned
data = np.genfromtxt(url,
            dtype=[('ra','f8'),('dec','f8'),('vmag','f8'),('hr','i8'),('hd','i8')],
            skip_header=34, 
            delimiter='\t')

# print ra,dec columns
print (data['ra'], data['dec'])

[ 25.914167  62.164583  84.410833 239.547083 246.755417 336.31875
 345.479583] [ 50.688889  47.713056  21.142778 -14.279167 -18.455833   1.377222
  42.326111]


### Using cache 
*cache* is a process to optimize resources access with saving temporary data into memory or on the file system. 

For example, browser uses cache to get internet pages on the client computer. It avoids querying the same URL when repeated.

Numpy provide the *DataSource* function to cache a file, an HTTP or FTP resources.

*DataSource* is used by the function *np.genfromtxt*

#### Delete the cache
DataSource creates a file on the current directory.

**Example: ** URL = http://vizier.u-strasbg.fr/viz-bin/asu-tsv?-source=2220/stars&-oc.form=dec&-out=Vmag,HR,HD&-out.add=_RA,_DE

&Rarr; Datasource create the file vizier.u-strasbg.fr/viz-bin/asu-tsv    

**Note** that the URL parameters (following '?') are not used to get a name. So the cache won't work if only a parameter changed!

In [9]:
import os

url = "vizier.u-strasbg.fr/viz-bin/asu-tsv"
arg = '-source=2220/stars&-oc.form=dec&-out=Vmag,HR,HD&-out.add=_RA,_DE'
try:
    os.unlink(url)
except:
    print ("no cache")

---------------------------------------------------------
### Astropy
![Image of Astropy](https://www.astropy.org/images/astropy_project_logo.svg)

- Astropy is based on Numpy (it includes some Numpy functionalities) 
- Astropy is licensed under a 3-clause BSD 
- Works with standardized format like:
    - FITS (includes PyFits)
    - VOTable (XL format used in the Virtual Observatory)
    - TSV- Ascii tables
    - HDF5...
- Usefull libraries  like
    - **Table and Data**
    - WCS (World Coordiante System): images/spectra projections
    - Time conversion
    - Cosmological Calculations (astropy.cosmology)
    - Data Visualization (astropy.visualization)
    - ...


### Table manipulation
Astropy API provides a user-firiendly interface to work with tables. It is based on Numpy but it simplifies the (fastidious) table usage.

Example  of functionalities (in package astropy.table)
- adding, removing columns
- enable sort and group by (like SQL)
- join tables according to criteria
…
See http://docs.astropy.org/en/stable/table/

#### First examples

Create an astropy table with named columns.

In [10]:
from astropy.table import Table

In [11]:
# create a table
table = Table([[0.000899, 0.004265, 0.005024], 
               [1.089009, -19.498840, 38.859279], 
               ['HIP1','HIP2', 'HIP3']],
              names=("ra", "dec", "name"))

# print the table
print (table)
print (table["name"]) # print the column "name"
print (table[0]) # print the first record

   ra       dec    name
-------- --------- ----
0.000899  1.089009 HIP1
0.004265 -19.49884 HIP2
0.005024 38.859279 HIP3
name
----
HIP1
HIP2
HIP3
   ra      dec    name
-------- -------- ----
0.000899 1.089009 HIP1


#### Create an astropy table with Numpy

In [12]:
import numpy as np

# create 3 columns using Numpy 
ra = np.array([0.000899, 0.004265, 0.005024])
dec = np.array([1.089009, -19.498840, 38.859279])
name = np.array(['HIP1','HIP2', 'HIP3'])

# Create Astropy table with name and type specified
table = Table([ra, dec, name], 
              names=("ra","dec","name"), 
              dtype=("f4","f4","S10"))

# print the table
print (table)

   ra       dec    name
-------- --------- ----
0.000899  1.089009 HIP1
0.004265 -19.49884 HIP2
0.005024  38.85928 HIP3


#### adding/removing columns

In [13]:
from astropy.table import Column

newcol = Column([9.1, 9.27, 6.61], name="Vmag")
table.add_column(newcol)
table.remove_column("name")
print (table)

   ra       dec    Vmag
-------- --------- ----
0.000899  1.089009  9.1
0.004265 -19.49884 9.27
0.005024  38.85928 6.61


#### Merge 2 tables

In [14]:
table2 = Table([[3.54, 21.9, 2.81], [9.643, 10.519, 6.576]], 
               names=("plx", "btmag"))

mytable = Table([table["ra"], table["dec"], table["Vmag"], table2["plx"], table2["btmag"]])
print (mytable)

   ra       dec    Vmag plx  btmag 
-------- --------- ---- ---- ------
0.000899  1.089009  9.1 3.54  9.643
0.004265 -19.49884 9.27 21.9 10.519
0.005024  38.85928 6.61 2.81  6.576


#### Modify values

In [15]:
table["Vmag"] *= 0.001
print (table)
print (mytable) # mytable is not affected by the modification

mytable["Vmag"] *= 0.001
print (mytable)

   ra       dec      Vmag 
-------- --------- -------
0.000899  1.089009  0.0091
0.004265 -19.49884 0.00927
0.005024  38.85928 0.00661
   ra       dec    Vmag plx  btmag 
-------- --------- ---- ---- ------
0.000899  1.089009  9.1 3.54  9.643
0.004265 -19.49884 9.27 21.9 10.519
0.005024  38.85928 6.61 2.81  6.576
   ra       dec      Vmag  plx  btmag 
-------- --------- ------- ---- ------
0.000899  1.089009  0.0091 3.54  9.643
0.004265 -19.49884 0.00927 21.9 10.519
0.005024  38.85928 0.00661 2.81  6.576


### Working with local (file) or remote resources
 
Astropy understand the common format like FITS, VOTable, TSV, HTML, LaTeX ...

- FITS and VOTable format contains the metadata that describes the data. 
- TSV files is an ascii file without standardized columns description. However, astropy can extract  metadata for some TSV format (example VizieR TSV which contains a header with comments)

See the format avaialble at: http://docs.astropy.org/en/stable/io/unified.html

In [16]:
# get a remote table (HTTP) with an ascii format
url = 'http://vizier.u-strasbg.fr/viz-bin/asu-tsv?-source=II/289/refs.dat'
table = Table.read(url, format='ascii.tab')

# print the 5 first lines
print (table[:5])

# print the Ref columns
print (table['Ref']) # column Jmag (if recognize in TSV header)

 Ref        BibCode       ...               Com               
----- ------------------- ... --------------------------------
                          ...                               --
----- ------------------- ... --------------------------------
A02   2002ApJ...566..993A ...                               --
A97   1997A&A...326..632A ...                               --
B03   2003ApJ...591.1064B ...                               --
 Ref 
-----
     
-----
A02  
A97  
B03  
B89  
C00  
C88  
CTK00
D04  
  ...
HM95 
LLR97
LR99 
M98  
N02  
S06  
SR49 
W05  
W94  
WGM99
WMG07
Length = 33 rows


An other example which uses the CDS format (ascii tables with aligned columns +ReadMe description)

In [17]:
table2 = Table.read("http://cdsarc.u-strasbg.fr/vizier/ftp/cats/II/289/refs.dat",
    format="ascii.cds",
    readme="http://cdsarc.u-strasbg.fr/vizier/ftp/cats/II/289/ReadMe")

print (table2[:2])

Ref       BibCode             Aut       Com
--- ------------------- --------------- ---
A02 2002ApJ...566..993A    Allen et al.  --
A97 1997A&A...326..632A Ageorges et al.  --


#### Print headers

In [18]:
# Get table (VizieR catalogue I/298 limited to 500 records) in VOTable format
table = tab = Table.read("http://vizier.u-strasbg.fr/viz-bin/votable?-source=I/298/lspm_n&-out.max=500")

# print the 5 first lines
print (tab[0:5])

   LSPM     NLTT TYC1   USNO-B1     RAJ2000   ...      pm     Aflag Vemag   V-J 
                                      deg     ... arcsec / yr        mag    mag 
---------- ----- ---- ------------ ---------- ... ----------- ----- ------ -----
J0000+0003 58703   -- 0900-0000025   0.075275 ...       0.253     S  15.02  3.27
J0000+0041 58724   -- 0906-0000116   0.129708 ...       0.198     S  16.31  3.70
J0000+0313 58739   -- 0932-0000102   0.168501 ...       0.352     S  17.43  3.72
J0000+0356 58690   -- 0939-0000011   0.018693 ...       0.227     A  12.39  2.58
J0000+0409    --    4 0941-0000118   0.211799 ...       0.170     T  11.71  1.78


Modify the verbosity because the VizieR VOTble output generates warnings

In [19]:
from astropy import log
log.setLevel('ERROR')


In [20]:
# Print metadata of Vemag column
print (table["Vemag"].meta)
print (table["Vemag"].description)

OrderedDict([('width', 6), ('precision', '2'), ('values', {'null': nan})])
? Estimated V magnitude (10)


#### Print table in a web page

In [24]:
table.show_in_browser(jsviewer=True) 

#### Add metadata

In [21]:
print (table["Vemag"].meta)

tab["Vemag"].meta["my_additional_description"]="Vemag column"
print (tab["Vemag"].meta)

OrderedDict([('width', 6), ('precision', '2'), ('values', {'null': nan})])
OrderedDict([('width', 6), ('precision', '2'), ('values', {'null': nan}), ('my_additional_description', 'Vemag column')])


### Make a sub-selection using astropy and Numpy

In [22]:
import numpy as np
ind = np.where(table["V-J"]<2)
print (table[ind])

    LSPM     NLTT TYC1   USNO-B1    ...      pm     Aflag Vemag   V-J 
                                    ... arcsec / yr        mag    mag 
----------- ----- ---- ------------ ... ----------- ----- ------ -----
 J0000+0409    --    4 0941-0000118 ...       0.170     T  11.71  1.78
 J0000+1659 58751 1178 1069-0000132 ...       0.313     T   8.80  1.85
 J0000+1758 58731 1181 1079-0000209 ...       0.365     T  10.60  1.98
 J0000+1906    --   -- 1091-0000108 ...       0.163     S  18.90  0.04
 J0000+2002 58692 1184 1100-0000025 ...       0.288     T   9.48  1.46
 J0000+3411 58691 2267 1241-0000033 ...       0.236     T   8.50  1.25
 J0000+3531    -- 2267 1255-0000040 ...       0.182     T  12.59  1.60
 J0000+3545    -- 2271 1257-0000228 ...       0.167     T  10.00  1.96
 J0000+3902 58700 2781 1290-0000077 ...       0.227     T  10.71  1.45
 J0000+4312 58705 2793 1332-0000149 ...       0.212     T  11.65  1.73
        ...   ...  ...          ... ...         ...   ...    ...   ...
 J0009

/usr/lib64/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


direct sub-selection using astropy

In [28]:
mask = (table["Vemag"] <10)
print(table[mask])

    LSPM     NLTT TYC1   USNO-B1    ...      pm     Aflag Vemag   V-J 
                                    ... arcsec / yr        mag    mag 
----------- ----- ---- ------------ ... ----------- ----- ------ -----
 J0000+1659 58751 1178 1069-0000132 ...       0.313     T   8.80  1.85
 J0000+2002 58692 1184 1100-0000025 ...       0.288     T   9.48  1.46
 J0000+3411 58691 2267 1241-0000033 ...       0.236     T   8.50  1.25
 J0001+2753 58761 1735 1178-0000469 ...       0.222     T   9.57  1.47
 J0001+7738    -- 4496 1676-0000281 ...       0.158     T   8.75  1.74
 J0002+2408    20 1729 1141-0000706 ...       0.203     T   9.16  1.18
 J0002+2704 58829 1732 1170-0000649 ...       1.291     T   5.77  1.07
 J0002+3706    -- 2271 1271-0000650 ...       0.159     T   8.80  1.31
 J0003+2039    -- 1184 1106-0000790 ...       0.168     T   7.49  1.17
 J0003+5643    24 3660 1467-0002175 ...       0.181     T   9.81  1.66
        ...   ...  ...          ... ...         ...   ...    ...   ...
 J0008

#### modify the units
convert column "V-J" in mmag

In [27]:
from astropy import units as u

table["V-J"].convert_unit_to(u.mmag)
print (table["V-J"][:5])

  V-J  
  mmag 
-------
3270.00
3700.00
3720.00
2580.00
1780.00


/usr/lib64/python3.4/site-packages/astropy/table/column.py:618: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  new_unit, self.data, equivalencies=equivalencies)


#### add galactic coordinates

In [28]:
from astropy.coordinates import SkyCoord

pos = SkyCoord(table["RAJ2000"], table["DEJ2000"], frame='icrs')
print (pos.to_string('hmsdms')[:3])

gal = pos.transform_to('galactic')

#table.remove_column("l")
#table.remove_column("b")
table.add_column(Column(gal.l, name="l"))
table.add_column(Column(gal.b, name="b"))

print (table[:3])

['00h00m18.066s +00d03m21.7944s', '00h00m31.1299s +00d41m08.6172s', '00h00m40.4402s +03d13m42.6s']
   LSPM     NLTT   USNO-B1    ...   V-J         l             b       
                              ...   mmag       deg           deg      
---------- ----- ------------ ... ------- ------------- --------------
J0000+0003 58703 0900-0000025 ... 3270.00 96.5209629243 -60.1670035914
J0000+0041 58724 0906-0000116 ... 3700.00 97.1124909892 -59.6089812288
J0000+0313 58739 0932-0000102 ... 3720.00  99.001127599  -57.265648543


### Mask columns
we put to null all Vemag values less than 15

In [29]:
from astropy.table import MaskedColumn

b = MaskedColumn(table["Vemag"], 
                 mask=table["Vemag"]<15, 
                 name="Vemag")
table.remove_column("Vemag")
table.add_column(b)

print (table["LSPM","Vemag"][:10])

   LSPM    Vemag 
            mag  
---------- ------
J0000+0003  15.02
J0000+0041  16.31
J0000+0313  17.43
J0000+0356     --
J0000+0409     --
J0000+0812  18.86
J0000+0852  16.57
J0000+1121  16.86
J0000+1348  16.37
J0000+1353  16.78


/usr/lib64/python3.4/site-packages/astropy/table/column.py:928: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, oper)(other)


#### assign a value for null values

In [30]:
table["Vemag"].fill_value = 99.99
print (table.filled()[:10])

   LSPM     NLTT    USNO-B1    ...       l             b        Vemag 
                               ...      deg           deg        mag  
---------- ------ ------------ ... ------------- -------------- ------
J0000+0003  58703 0900-0000025 ... 96.5209629243 -60.1670035914  15.02
J0000+0041  58724 0906-0000116 ... 97.1124909892 -59.6089812288  16.31
J0000+0313  58739 0932-0000102 ...  99.001127599  -57.265648543  17.43
J0000+0356  58690 0939-0000011 ... 99.2192525065 -56.5420234981  99.99
J0000+0409 999999 0941-0000118 ... 99.6845915004 -56.4081323611  99.99
J0000+0812 999999 0982-0000215 ... 102.043116098 -52.5966966396  18.86
J0000+0852  58753 0988-0000105 ... 102.378684721 -51.9595456817  16.57
J0000+1121  58688 1013-0000009 ... 103.307483848 -49.5410120877  16.86
J0000+1348  58687 1038-0000015 ... 104.410371555 -47.2165500041  16.37
J0000+1353 999999 1038-0000126 ... 104.675946125 -47.1775200567  16.78


#### save result into a local file (format ascii CDS with aligned column)

In [31]:
table.write("mytable.ascii", format="ascii.fixed_width")

#### Back to original values

In [32]:
table.mask = False
print (table["LSPM","Vemag"][:10])

   LSPM    Vemag 
            mag  
---------- ------
J0000+0003  15.02
J0000+0041  16.31
J0000+0313  17.43
J0000+0356  12.39
J0000+0409  11.71
J0000+0812  18.86
J0000+0852  16.57
J0000+1121  16.86
J0000+1348  16.37
J0000+1353  16.78


### Join 2 tables
we join 2 tables comint gtom hipparcos catalogue

see http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=I/239/
- I/239/hip_main
- I/239/hp_refs

In [33]:
tableref = Table.read("http://vizier.u-strasbg.fr/viz-bin/votable?-source=I/239/hp_refs&-out.max=1000") 
table = Table.read("http://vizier.u-strasbg.fr/viz-bin/votable?-source=I/239/hip_main&-out.max=1000")

from astropy.table import join

# make the join according to the columln HIP
jointable = join(table, tableref, keys='HIP', join_type='outer')

# print the result
print (jointable)
print (jointable.columns)

HIP     RAhms       DEdms     Vmag ...   _DE.icrs   Ntot Nline  nRef 
                              mag  ...     deg                       
---- ----------- ----------- ----- ... ------------ ---- ----- ------
   1 00 00 00.22 +01 05 20.4  9.10 ...   1.08900875   --    --     --
   2 00 00 00.91 -19 29 55.8  9.27 ... -19.49883971   --    --     --
   3 00 00 01.20 +38 51 33.4  6.61 ...  38.85927901   --    --     --
   4 00 00 02.01 -51 53 36.8  8.06 ... -51.89354573   --    --     --
   5 00 00 02.39 -40 35 28.4  8.55 ... -40.59120235   --    --     --
   6 00 00 04.35 +03 56 47.4 12.31 ...   3.94645772   --    --     --
   7 00 00 05.41 +20 02 11.8  9.64 ...  20.03611413   --    --     --
   8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     3 89.031
   8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     4 88.020
   8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     5 70.009
 ...         ...         ...   ... ...          ...  ...   ...    ...
3886          --    

#### Different join types 
- *inner*: get records when records exist in the 2 tables
- *left*: get all records ot the left table  
- *right*: get all records of te right table
- *outer*:  get all records 

In [34]:
print (join(table, tableref, keys='HIP', join_type='inner'))

HIP    RAhms       DEdms     Vmag ...   _DE.icrs   Ntot Nline  nRef 
                             mag  ...     deg                       
--- ----------- ----------- ----- ... ------------ ---- ----- ------
  8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     3 89.031
  8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     4 88.020
  8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     5 70.009
  8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     6 22.006
  8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     7  7.001
  8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     2 95.692
  8 00 00 06.55 +25 53 11.3  9.05 ...  25.88646069    7     1 95.588
 14 00 00 11.59 -00 21 37.5  7.25 ...  -0.36044955    1     1 95.445
 19 00 00 12.80 +38 18 14.7  6.53 ...  38.30404973    1     1 94.406
 25 00 00 19.05 -44 17 25.1  6.28 ... -44.29056147    1     1 95.445
...         ...         ...   ... ...          ...  ...   ...    ...
951 00 11 44.75 +57 16 17.5  7.06 

### Working with FITS

![FITS nasa](https://fits.gsfc.nasa.gov/FITSlogo.gif)
FITS is a common format used in astronomy to store table, images or spectra. It looks like a container that can includes several resources (example: 1 image and 2 spectra) called HDU.

FITS is a binary format with:
- a main header describing the FITS files and the different resources (name, size, type..)
- a header per resources describing the data:
    - columns description for Table
    - WCS for spectra and images
- but also some information like the date of observation, the telescopes, comments, …

The FITS standards are described by the Nasa website: https://fits.gsfc.nasa.gov/

A common tools to explore FITS files is *fv* provided by the Nasa : https://heasarc.gsfc.nasa.gov/ftools/fv/

FV Snaphots:
![fv snapshot](FV.png)


#### Open a file with astropy

In [86]:
import astropy.io.fits 

In [87]:
# open a local file 'asyu.fits'
fit = astropy.io.fits.open('asu.fits') 

# print metadata information
fit.info()

# close the file
fit.close() 

Filename: asu.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      27   ()      
  1  II_246_out    1 TableHDU        75   18R x 15C   [F10.6, F10.6, A17, F6.3, F6.3, F6.3, F6.3, F6.3, F6.3, A3, A3, A3, A3, I1, I1]   


The object returned by astropy.io.fits.open is an array of objects – with one  object per resource (HDU).

**Example**: metadata extraction ot the 2d resources (HDU2)

In [88]:
# open FITS file
fit = astropy.io.fits.open('asu.fits') 

# get header and data
tablehdu = fit[1]

print ("Number of records: {}".format(tablehdu.header['NAXIS2']))
print ("Type de l'extension: {}".format(tablehdu.header['XTENSION']))

# print the first lines of the table 
print (tablehdu.data[0])

# close the FITS
fit.close()

Number of records: 18
Type de l'extension: TABLE
(10.439434, 89.977051000000003, '00414546+8958373', 15.359999999999999, 0.068000000000000005, 14.871, 0.10100000000000001, 14.606, 0.114, 'AAB', '222', '111', '000', 0, 0)


#### about FITS metadata
The FITS standard defines a list of standardized keywords for table, spectra images...

**Example:**

Name | Description
---- | ----
TELESCOP | the telescop 
DATEMIN | begining of the observation
DATEMAX | end of observation
COMMENT | comments
RA | right ascension
DEC | declination

positions or electromagnetics values are also described by standardized keywords in FITS with WCS.

Standard for Tables extension contains keywords to describe columns 

#### Extract FITS metadata


In [89]:
fit = astropy.io.fits.open('asu.fits')
columns=fit[1].columns

# print column name and unit
for col in columns:
    print ("name {}, unit {}".format(col.name, col.unit))
    
#print the columlns in an array
print (columns.names)

# close FITS
fit.close()

name RAJ2000, unit deg
name DEJ2000, unit deg
name _2MASS, unit None
name Jmag, unit mag
name e_Jmag, unit mag
name Hmag, unit mag
name e_Hmag, unit mag
name Kmag, unit mag
name e_Kmag, unit mag
name Qflg, unit None
name Rflg, unit None
name Bflg, unit None
name Cflg, unit None
name Xflg, unit None
name Aflg, unit None
['RAJ2000', 'DEJ2000', '_2MASS', 'Jmag', 'e_Jmag', 'Hmag', 'e_Hmag', 'Kmag', 'e_Kmag', 'Qflg', 'Rflg', 'Bflg', 'Cflg', 'Xflg', 'Aflg']


#### columns attributes
Name | Description
---- | ----
name | column name
format | (FORTRAN) format (e.g.:A3: String with length<=3, F5.2: double with 4 digits (+1  with '.') and 2 gigits after the point)
unit | units (deg, 'h:m:s', mag,..)
start, bscale, bzero, disp... | other attributes

#### Extract the table data

In [92]:
fit = astropy.io.fits.open('asu.fits')
data = fit[1].data

# print data line by line
for row in data:
    print (row) # print current line



(10.439434, 89.977051000000003, '00414546+8958373', 15.359999999999999, 0.068000000000000005, 14.871, 0.10100000000000001, 14.606, 0.114, 'AAB', '222', '111', '000', 0, 0)
(23.510793, 89.977363999999994, '01340259+8958385', 16.475000000000001, 0.17899999999999999, 15.874000000000001, 0.0, 16.140000000000001, 0.0, 'CUU', '200', '100', '000', 0, 0)
(74.304832000000005, 89.979911999999999, '04571315+8958476', 16.702999999999999, 0.13800000000000001, 17.574999999999999, 0.0, 16.318000000000001, 0.0, 'BUU', '200', '100', '000', 0, 0)
(78.158658000000003, 89.989044000000007, '05123807+8959205', 15.715, 0.076999999999999999, 15.013, 0.099000000000000005, 14.672000000000001, 0.094, 'AAA', '222', '111', '000', 0, 0)
(78.403255000000001, 89.985268000000005, '05133678+8959069', 18.600000000000001, 0.0, 16.478999999999999, 0.0, 15.077999999999999, 0.125, 'UUB', '002', '001', '000', 0, 0)
(79.221886999999995, 89.972487999999998, '05165325+8958209', 16.056000000000001, 0.127, 15.654, 0.216, 15.353, 

In [94]:
print (data[0].field(0)) # print the first value of the current line (first column)

# print the array of the first column
print (data.field(0))
# print the arary of the column RAJ2000
print (data.field("RAJ2000"))


fit.close()

10.439434
[  10.439434   23.510793   74.304832   78.158658   78.403255   79.221887
   86.793746  110.92212   180.216382  191.63972   197.450988  241.847897
  260.073455  304.956115  316.795845  321.653808  342.747968  347.869091]
[  10.439434   23.510793   74.304832   78.158658   78.403255   79.221887
   86.793746  110.92212   180.216382  191.63972   197.450988  241.847897
  260.073455  304.956115  316.795845  321.653808  342.747968  347.869091]


#### Create a FITS file


In [96]:
import astropy.io.fits as fits
import numpy 

# create numpy arrays 
name = numpy.array(['NGC1001', 'NGC1002', 'NGC1003'])
mag = numpy.array([11.1, 12.3, 15.2])

# create FITS column object from previous numpay array
colname = astropy.io.fits.Column(name='target', format='20A', array=name)
colmag = astropy.io.fits.Column(name='V_mag', format='E', array=mag)

# Create a Table extension (HDU)
table = fits.BinTableHDU.from_columns([colname, colmag])
table.writeto('table2.fits')

**Notes:** astropy.Table package provides also functionalities to create a FITS files

In [47]:
table = Table([name, mag], names=("target", "V_mag"))
table.write("mytable.fits", format="fits")

#### Open FITS  using a URL


In [97]:
url="http://vizier.u-strasbg.fr/viz-bin/asu-fits?-source=2220/stars"
fit = fits.open(url) 
print (fit.info())
fit.close()

Filename: /home/cds/.astropy/cache/download/py3/697eef75a1831bb9075933461ff15451
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      30   ()      
  1  II_220_stars    1 TableHDU        51   10R x 10C   [A1, A7, I6, I4, F5.2, A10, A9, A15, I3, A6]   
None


** Note: **astropy provides also functionalities for VOTable 

### Working on FITS image

- FITS file is a container which can have several image extension (or HDU).
- 2D Images are stored in a *pixel* table (2x2)
- FITS header (should) contain metadata which enables to get the position of the image. FITS standard use WCS (World Coordinate System) for the projection (projection from a map to the spehere).

WCS Keywords :

Name | Description
---- | ----
NAXIS | number of axes (=2 for 2D images)
CTYPEx | projection name for x-axis (e.g.: tangenatial projection: RA--TAN )
CRVALx | (double) reference value for the x-axis
CDELTx | gap between 2 pixel  for the x-axis
CRPIXx | reference pixel for x-axis
CD_i_j | rotation matrix 


**Note:** Spectrum WCS is similar than images with other projections and in 1D.

### Example: extraction of a subpart of an image
We search the position of the center of the image

![fits extraction](./fits_extraction.png)

In [3]:
import astropy.io.fits
import astropy.wcs 

url = "http://cdsarc.u-strasbg.fr/saadavizierlocal/download?oid=864973093058118649" 
#url = "http://cdsarc.u-strasbg.fr/saadavizier/download?oid=864972989978902529"

# get the extraction
fit = astropy.io.fits.open(url)
# get first HDU(=extension)
hdu = fit[0] 
size = (hdu.header["NAXIS1"], hdu.header["NAXIS2"])
print ("size=({},{})".format(size[0], size[1]))
fit.close()

# get WCS in the header
wcs = astropy.wcs.WCS(url)

ra,de = wcs.wcs_pix2world(size[0], size[1], 0)
print ("position : {}, {}".format(ra,de))

size=(720,720)
position : 721.0, 721.0


http://cdsarc.u-strasbg.fr/saadavizierlocal/download?oid=864973093058118649      [astropy.io.fits.card]


### Other example: update an image with thresholding
thresholding is an operation which put to null all values (pixel values) which are less than a constant.

![image after  threshold](./fits-threshold_1.png)
![image after  threshold](./fits-threshold_2.png)

In [ ]:
import astropy.io.fits
import astropy.wcs

Threshold = 0.4

# get image
url = "http://cdsarc.u-strasbg.fr/saadavizierlocal/download?oid=864973093058118649"
hdu = astropy.io.fits.open(url)

# make a copy 
newhdu = hdu[0].copy()

# read pixel array value by value and set to null if needed
naxis1 = hdu[0].header["NAXIS1"]
naxis2 = hdu[0].header["NAXIS2"]
for i in range(naxis1):
    for j in range(naxis2):
        if newhdu.data[j][i]<Threshold:
            
            newhdu.data[j][i]=0.
hdu.close()
newhdu.writeto("new.fits")

#### the same program using numpy (more efficient!)

In [ ]:
import astropy.io.fits
import astropy.wcs
import numpy

Threshold = 0.4

# get image
url = "http://cdsarc.u-strasbg.fr/saadavizierlocal/download?oid=864973093058118649"
hdu = astropy.io.fits.open(url)
newhdu = hdu[0].copy()
newhdu.data = numpy.where(hdu[0].data > Threshold, hdu[0].data, 0)

hdu.close()
newhdu.writeto("new.fits")

------------------------------------------
## TD1: query Gaia+hipparcos catalogue using astropy

1. create a function to get Gaia data (table tgas) from VizieR around a position given in parameter using *astropy.table* (format VOTable)

    Web site : http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=I/337

    VOTable url : http://vizier.u-strasbg.fr/viz-bin/votable?-source=I/337/tgas&-c={}&-c.rd=2&-c.eq=Galactic
    
    where {} is the position , ex: M1, M2, HD10..

2. update the function with curation :

    extract data with propper motion <10 : 

    pmRA\*pmRA+pmDE\*pmDE<10 (use numpy.where)

3. create a function to get Hipparcos data from VizieR around a position given in parameter

    Web site : http://vizier.u-strasbg.fr/viz-bin/VizieR?-source=I/239
    
    VOTable url: http://vizier.u-strasbg.fr/viz-bin/votable?-source=I/239/hip_main&-c={}&-c.rd=2&-c.eq=Galactic
    
    where {} is the position , ex: M1, M2, HD10..

4. update the function with curation :

    Build an astropy table containing only records having a note (='P'): 

    constraint: table['Notes']=='P'

5. query gaia and hipparcos using the previous function for the position 0+0

6. join the 2 tables using the HIP column 
  
   Note: extract before gaia data having HIP number : constraint gaia['HIP']>0

## TD2: compute color in hiparcos catalogue (around M31)

1. get hipparcos data from VizieR acound the galactic center using astropy: (format FITS)
    
    FITS url: http://http://vizier.cfa.harvard.edu/viz-bin/asu-fits?-source=I/239/hip_main&-c=0+0&-c.rd=2&-c.eq=Galactic&-out=RAICRS,DE_ICRS,HIP,BTmag,VTmag
    
2. compute the color BTmag-VTMag into a new columns 'BV' that you add into a new astropy.table.Table

3. fill BV columns values with -999  when the value BV>1

4. store the result into a fits table